In [1]:
# import dependencies
import pandas as pd
import gmaps
from sqlalchemy import create_engine
from config import api_key
from urllib import response
import requests
import json
import numpy as np

### store CSV data into a DataFrame
##### In this project, we were keen to evaluate Airbnb data sourced from Kaggle. Airbnb datasets were downloaded in a CSV format, read into pandas and cleaned to have relevant dataframes

In [2]:
# read the csv into pandas
listing_df=pd.read_csv('resources/listings.csv')
listing_df.head()
len(listing_df)

20420

In [8]:
# select the most desired columns to be stored in sql databse
# perform necessary cleaning routines
# due to the dataset being large, the team opted for top 100 airbnb's in melbourne
new_df=listing_df[['id', 'host_id', 'name', 'host_location', 'review_scores_location', \
    'latitude', 'longitude', 'price']]
new_df.rename(columns={'name':'airbnb_name'}, inplace=True)
new_df.sort_values(by=['review_scores_location'], ascending=False)
new_df=new_df.head(100)
new_df.head()

/Users/russ/opt/anaconda3/envs/bootcamp/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,id,host_id,airbnb_name,host_location,review_scores_location,latitude,longitude,price
0,9835,33057,Beautiful Room & House,AU,9.0,-37.77268,145.09213,$60.00
1,10803,38901,Room in Cool Deco Apartment in Brunswick East,"Melbourne, Victoria, Australia",10.0,-37.76651,144.98074,$31.00
2,12936,50121,St Kilda 1BR+BEACHSIDE+BALCONY+WIFI+AC,"Melbourne, Victoria, Australia",10.0,-37.85999,144.97662,$95.00
3,38271,164193,Melbourne - Old Trafford Apartment,"Berwick, Victoria, Australia",10.0,-38.05725,145.33936,$99.00
4,41836,182833,CLOSE TO CITY & MELBOURNE AIRPORT,"Melbourne, Victoria, Australia",9.0,-37.69729,145.00082,$40.00


In [5]:
# read in the csv dataset
reviews_df=pd.read_csv('resources/reviews.csv', encoding='utf-8')
reviews_df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,9835,279854,2011-05-24,560832,Miriam,"Very hospitable, much appreciated.\r\n"
1,9835,3640746,2013-02-26,5143343,Michelle,A beautiful house in a lovely quiet neighbourh...
2,9835,23731188,2014-12-08,2478713,Karyn,This was my first time using airbnb and it was...
3,9835,46588875,2015-09-12,26184717,Rosalind,I was visiting Melbourne to spend time with my...
4,10803,3333588,2013-01-12,4421189,Johannes,It was very convenient to stay in Lindsay's a...


In [6]:
# extracted the needed columns to be stored in sql database
new_df3=reviews_df[['listing_id', 'id', 'comments']]
new_df3.head()

,listing_id,id,comments
0,9835,279854,"Very hospitable, much appreciated.\r\n"
1,9835,3640746,A beautiful house in a lovely quiet neighbourh...
2,9835,23731188,This was my first time using airbnb and it was...
3,9835,46588875,I was visiting Melbourne to spend time with my...
4,10803,3333588,It was very convenient to stay in Lindsay's a...


### extract data from Google API 
##### This provided restaurant information near the targeted Airbnb locations. Also cleaned and formatted for further usage.
##### Reason for extracting this data was to enable further analysis on where Airbnb guest might like to dine from or order takeouts from, the group thought this would be of keen interests to guests and hosts.

In [9]:
# create a new dataframe to append extracted restaurant details
# notice we have empty columns which will be filled after parsing in the API
# maintained 'id' column to establish relationship in sql database as primary key
cafe_df=new_df[['id','airbnb_name','latitude','longitude']]
cafe_df['cafe_name']=''
cafe_df['cafe_rating']=''
cafe_df

/Users/russ/opt/anaconda3/envs/bootcamp/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/russ/opt/anaconda3/envs/bootcamp/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,airbnb_name,latitude,longitude,cafe_name,cafe_rating
0,9835,Beautiful Room & House,-37.77268,145.09213,,
1,10803,Room in Cool Deco Apartment in Brunswick East,-37.76651,144.98074,,
2,12936,St Kilda 1BR+BEACHSIDE+BALCONY+WIFI+AC,-37.85999,144.97662,,
3,38271,Melbourne - Old Trafford Apartment,-38.05725,145.33936,,
4,41836,CLOSE TO CITY & MELBOURNE AIRPORT,-37.69729,145.00082,,
...,...,...,...,...,...,...
95,449585,Need Luxury Homebase in Melbourne 1,-37.80491,145.05325,,
96,453940,Corner room - Private & Lovely Balwyn Nth Home,-37.79559,145.07892,,
97,456950,St Kilda Bay Views,-37.85397,144.98198,,
98,464528,BOUTIQUE STAYS – Zinc Views - 501,-37.84318,144.94016,,


In [10]:
# params dictionary to update
params={
    'radius':20,
    'types':'restaurant',
    'keyword':'restaurant',
    'key':api_key
}

# use coordinates from cafe_db dataframe
for index, row in cafe_df.iterrows():
    lat=row['latitude']
    lng=row['longitude']

    params['location']=f'{lat},{lng}'

    url='https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    response=requests.get(url, params=params).json()

    # print(json.dumps(response, indent=4, sort_keys=True))

    try:
        cafe_df.loc[index,'cafe_name']=response['results'][0]['name']
        cafe_df.loc[index, 'cafe_rating']=response['results'][0]['rating']

    except(KeyError, IndexError):
        print('stay put as we keep searching for a nearby cafe')


/Users/russ/opt/anaconda3/envs/bootcamp/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


stay put as we keep searching for a nearby cafe
stay put as we keep searching for a nearby cafe
stay put as we keep searching for a nearby cafe
stay put as we keep searching for a nearby cafe
stay put as we keep searching for a nearby cafe
stay put as we keep searching for a nearby cafe
stay put as we keep searching for a nearby cafe
stay put as we keep searching for a nearby cafe
stay put as we keep searching for a nearby cafe
stay put as we keep searching for a nearby cafe
stay put as we keep searching for a nearby cafe
stay put as we keep searching for a nearby cafe
stay put as we keep searching for a nearby cafe
stay put as we keep searching for a nearby cafe
stay put as we keep searching for a nearby cafe
stay put as we keep searching for a nearby cafe
stay put as we keep searching for a nearby cafe
stay put as we keep searching for a nearby cafe
stay put as we keep searching for a nearby cafe
stay put as we keep searching for a nearby cafe
stay put as we keep searching for a near

In [11]:
# clean the extracted data in the cafe df
cafe_df['cafe_name'].replace('', np.nan, inplace=True)
# cafe_df['cafe rating'].replace('', np.nan, inplace=True)
cafe_df.dropna(subset=['cafe_name'], inplace=True)
cafe_df

/Users/russ/opt/anaconda3/envs/bootcamp/lib/python3.7/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/Users/russ/opt/anaconda3/envs/bootcamp/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,id,airbnb_name,latitude,longitude,cafe_name,cafe_rating
0,9835,Beautiful Room & House,-37.77268,145.09213,Mulino Woodfire Pizzeria,4.6
2,12936,St Kilda 1BR+BEACHSIDE+BALCONY+WIFI+AC,-37.85999,144.97662,Leo's Spaghetti Bar,4.1
3,38271,Melbourne - Old Trafford Apartment,-38.05725,145.33936,Masala Bar and Grill,3.7
5,43429,Tranquil Javanese-Style Apartment in Oakleigh ...,-37.89960,145.11447,Tarantino's Pizzeria,4.4
7,47100,Très Charming in Fabulous Richmond,-37.81804,145.00442,Bouzy Rouge,4.4
8,51592,Central City Warehouse Apartment,-37.81486,144.96267,Tipo 00,4.6
11,67211,"Kew Tranquility, Melbourne",-37.80370,145.03396,Dawson Eatery & Bar,4.5
14,70328,AWESOME BEACH HOUSE BnB -,-38.00546,145.08541,BANG BANG,4.4
15,72576,Stunning Fitzroy +private bathroom 'Little Geo...,-37.80689,144.97946,Trippy Taco,4.3
16,74324,Tamerlane: Architect-designed luxury living,-37.80275,144.97616,Bon Ap' Petit Bistro,4.6


### Connecting to local database

In [15]:
connection_string='postgres:Harvey90@localhost:5432/etl_project'
engine=create_engine(f'postgresql://{connection_string}')

In [16]:
# confirm tables
engine.table_names()

/Users/russ/opt/anaconda3/envs/bootcamp/lib/python3.7/site-packages/ipykernel_launcher.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  


['listing', 'reviews', 'cafes']

In [18]:
new_df.to_sql(name='listing', con=engine, if_exists='append', index=False)

In [19]:
cafe_df.to_sql(name='cafes', con=engine, if_exists='append', index=False)

In [20]:
new_df3.to_sql(name='reviews', con=engine, if_exists='append', index=False)

In [21]:
pd.read_sql_query('select * from listing', con=engine).head()

,id,host_id,airbnb_name,host_location,review_scores_location,latitude,longitude,price
0,9835,33057,Beautiful Room & House,AU,9.0,-37.77268,145.09213,$60.00
1,10803,38901,Room in Cool Deco Apartment in Brunswick East,"Melbourne, Victoria, Australia",10.0,-37.76651,144.98074,$31.00
2,12936,50121,St Kilda 1BR+BEACHSIDE+BALCONY+WIFI+AC,"Melbourne, Victoria, Australia",10.0,-37.85999,144.97662,$95.00
3,38271,164193,Melbourne - Old Trafford Apartment,"Berwick, Victoria, Australia",10.0,-38.05725,145.33936,$99.00
4,41836,182833,CLOSE TO CITY & MELBOURNE AIRPORT,"Melbourne, Victoria, Australia",9.0,-37.69729,145.00082,$40.00


In [22]:
pd.read_sql_query('select * from reviews', con=engine).head()

,id,listing_id,comments
0,279854,9835,"Very hospitable, much appreciated.\r\n"
1,3640746,9835,A beautiful house in a lovely quiet neighbourh...
2,23731188,9835,This was my first time using airbnb and it was...
3,46588875,9835,I was visiting Melbourne to spend time with my...
4,3333588,10803,It was very convenient to stay in Lindsay's a...


In [23]:
pd.read_sql_query('select * from cafes', con=engine).head()

,id,airbnb_name,latitude,longitude,cafe_name,cafe_rating
0,9835,Beautiful Room & House,-37.77268,145.09213,Mulino Woodfire Pizzeria,4.6
1,12936,St Kilda 1BR+BEACHSIDE+BALCONY+WIFI+AC,-37.85999,144.97662,Leo's Spaghetti Bar,4.1
2,38271,Melbourne - Old Trafford Apartment,-38.05725,145.33936,Masala Bar and Grill,3.7
3,43429,Tranquil Javanese-Style Apartment in Oakleigh ...,-37.89960,145.11447,Tarantino's Pizzeria,4.4
4,47100,Très Charming in Fabulous Richmond,-37.81804,145.00442,Bouzy Rouge,4.4
